In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
df = pd.read_csv('C:/users/igorv/MATHMOD/Coursera/US Police shootings in from 2015-22.csv', sep = ',')
df

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,longitude,latitude,is_geocoding_exact
0,1,Tim Elliot,2015-01-02,shot,gun,53.0,M,A,Shelton,WA,True,attack,Not fleeing,False,-123.122,47.247,True
1,2,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False,-122.892,45.487,True
2,3,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False,-97.281,37.695,True
3,4,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False,-122.422,37.763,True
4,5,Michael Rodriguez,2015-01-04,shot,nail gun,39.0,M,H,Evans,CO,False,attack,Not fleeing,False,-104.692,40.384,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7677,7678,NaN,2022-08-23,shot,unknown weapon,NaN,M,NaN,Harris County,TX,False,other,Foot,False,-95.479,29.963,True
7678,7679,NaN,2022-08-23,shot,machete,48.0,M,NaN,San Diego,CA,False,other,Not fleeing,False,-117.108,32.723,True
7679,7680,Geraldo Reyes,2022-08-23,shot,gun,32.0,M,NaN,Cut Off,LA,False,attack,NaN,False,-90.312,29.458,True
7680,7681,NaN,2022-08-23,shot,gun,NaN,M,NaN,Blue Springs,MO,False,attack,Foot,False,-94.258,39.023,True


In [3]:
# lets make table with only age and gender, where we will make Male = 1, Female = 0
df2 = df[['age','gender', 'body_camera', 'signs_of_mental_illness']].dropna()
# after clearing dataset, lets make index column comfortable 
df2.index = np.arange(1, len(df2) + 1) 
df2

,age,gender,body_camera,signs_of_mental_illness
1,53.0,M,False,True
2,47.0,M,False,False
3,23.0,M,False,False
4,32.0,M,False,True
5,39.0,M,False,False
...,...,...,...,...
7200,37.0,M,False,False
7201,42.0,F,False,False
7202,41.0,F,False,False
7203,48.0,M,False,False


In [4]:
df3 = df[['age','gender', 'body_camera','race', 'signs_of_mental_illness']].dropna()
df3

,age,gender,body_camera,race,signs_of_mental_illness
0,53.0,M,False,A,True
1,47.0,M,False,W,False
2,23.0,M,False,H,False
3,32.0,M,False,W,True
4,39.0,M,False,H,False
...,...,...,...,...,...
7585,23.0,M,True,B,False
7600,37.0,M,False,W,True
7622,27.0,M,True,W,False
7654,23.0,M,False,W,False


In [5]:
df3['gender'] = np.where((df3.gender == 'F'), 0, df3.gender )
df3['gender'] = np.where((df3.gender == 'M'), 1, df3.gender )

df3['body_camera'] = np.where((df3.body_camera == 'False'), 0, df3.body_camera )
df3['body_camera'] = np.where((df3.body_camera == 'True'), 1, df3.body_camera )

df3['signs_of_mental_illness'] = np.where((df3.signs_of_mental_illness == 'True'), 1, df3.signs_of_mental_illness )
df3['signs_of_mental_illness'] = np.where((df3.signs_of_mental_illness == 'False'), 0, df3.signs_of_mental_illness )

In [6]:
df3.reset_index()

,index,age,gender,body_camera,race,signs_of_mental_illness
0,0,53.0,1,0,A,1
1,1,47.0,1,0,W,0
2,2,23.0,1,0,H,0
3,3,32.0,1,0,W,1
4,4,39.0,1,0,H,0
...,...,...,...,...,...,...
6116,7585,23.0,1,1,B,0
6117,7600,37.0,1,0,W,1
6118,7622,27.0,1,1,W,0
6119,7654,23.0,1,0,W,0


In [7]:
df3.dtypes

age                        float64
gender                      object
body_camera                  int32
race                        object
signs_of_mental_illness      int32
dtype: object

In [8]:
lreg1 = smf.ols('age ~ gender + body_camera + signs_of_mental_illness + C(race)', data = df3).fit()
print(lreg1.summary())

                            OLS Regression Results                            
Dep. Variable:                    age   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     66.12
Date:                Tue, 11 Oct 2022   Prob (F-statistic):          1.65e-104
Time:                        21:03:47   Log-Likelihood:                -23957.
No. Observations:                6121   AIC:                         4.793e+04
Df Residuals:                    6112   BIC:                         4.799e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [9]:
lreg1 = smf.logit(formula = 'body_camera ~ signs_of_mental_illness + gender + age', data = df3).fit()
print(lreg1.summary())

Optimization terminated successfully.
         Current function value: 0.409537
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:            body_camera   No. Observations:                 6121
Model:                          Logit   Df Residuals:                     6117
Method:                           MLE   Df Model:                            3
Date:                Tue, 11 Oct 2022   Pseudo R-squ.:                0.006063
Time:                        21:03:47   Log-Likelihood:                -2506.8
converged:                       True   LL-Null:                       -2522.1
Covariance Type:            nonrobust   LLR p-value:                 1.042e-06
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  -1.4738      0.205     -7.181      0.000      -1.876   

In [10]:
params = lreg1.params
conf = lreg1.conf_int()
conf['OR'] = params
conf.columns = ['Lower CI', 'Upper CI', 'OR']
print(np.exp(conf))

                         Lower CI  Upper CI        OR
Intercept                0.153200  0.342480  0.229059
gender[T.1]              0.781352  1.562708  1.105000
signs_of_mental_illness  1.160674  1.613269  1.368386
age                      0.980915  0.992643  0.986762
